Positional Encoding

In [1]:
import numpy as np
import math
def positional_encoding(max_len, d_model):
    # max_len: độ dài tối đa của 1 chuỗi
    # d_model: kích thước embedding
    pe = np.zeros((max_len, d_model))
    for pos in range(max_len):
        for i in range(0, d_model, 2):
            angle = pos / (10000 ** (i / d_model))
            # Vị trí chẵn
            pe[pos, i] = math.sin(angle)
            # Vị trí lẻ
            pe[pos, i+1] = math.cos(angle)
    return pe



In [2]:
max_len = 5
d_model = 4
pe = np.zeros((5, 4))
print(f"ma trận ban đầu:\n{pe}")
print(f"------------------")
for pos in range(max_len):
    for i in range(0, d_model, 2):
        angle = pos / (10000 ** (i / d_model))
        pe[pos][i] = math.sin(angle)
        pe[pos][i+1] = math.cos(angle)
print(f"ma trận positional encoding:\n{pe}")
print(f"--------------------------")
for i in range(max_len):
    print(f"vector PE cho vị trí thứ: {i} là:\n{pe[i]}")
    print(f"------------------------------")


ma trận ban đầu:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
------------------
ma trận positional encoding:
[[ 0.          1.          0.          1.        ]
 [ 0.84147098  0.54030231  0.00999983  0.99995   ]
 [ 0.90929743 -0.41614684  0.01999867  0.99980001]
 [ 0.14112001 -0.9899925   0.0299955   0.99955003]
 [-0.7568025  -0.65364362  0.03998933  0.99920011]]
--------------------------
vector PE cho vị trí thứ: 0 là:
[0. 1. 0. 1.]
------------------------------
vector PE cho vị trí thứ: 1 là:
[0.84147098 0.54030231 0.00999983 0.99995   ]
------------------------------
vector PE cho vị trí thứ: 2 là:
[ 0.90929743 -0.41614684  0.01999867  0.99980001]
------------------------------
vector PE cho vị trí thứ: 3 là:
[ 0.14112001 -0.9899925   0.0299955   0.99955003]
------------------------------
vector PE cho vị trí thứ: 4 là:
[-0.7568025  -0.65364362  0.03998933  0.99920011]
------------------------------


Sau khi tính PE, ta cộng vào embedding ban đầu: X' = X + PE

In [3]:
# Giả sử vector X ban đầu có giá trị:
x = np.random.randn(max_len, d_model)
print(f"vector embedding của input:\n{x}")
x = x + pe
print(f"---------------------------------")
print(f"vector x khi thêm positional encoding:\n{x}")

vector embedding của input:
[[ 0.89818637 -1.67199328  1.65248184 -0.75380082]
 [-0.36766704 -0.70585323  1.68249251 -1.3323367 ]
 [ 1.54309046 -0.63104485  0.49422014 -0.70144544]
 [-0.4908879  -0.76398075  0.22342576  0.23296773]
 [ 0.42854906 -0.84122937 -0.81525255  0.52525397]]
---------------------------------
vector x khi thêm positional encoding:
[[ 0.89818637 -0.67199328  1.65248184  0.24619918]
 [ 0.47380395 -0.16555093  1.69249234 -0.3323867 ]
 [ 2.45238789 -1.04719169  0.51421881  0.29835456]
 [-0.34976789 -1.75397325  0.25342126  1.23251776]
 [-0.32825343 -1.49487299 -0.77526321  1.52445408]]


Multihead Attention

Residual and Normalization Layer

In [ ]:
import torch
import torch.nn as nn
# Layer normalization
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-6):
        # gamma và beta
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zero(d_model))
        self.eps = eps
    def forward(self, x):
        # x shape: (batch_size, seq_len, d_model)
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased= False)

        x_hat = (x - mean) / torch.sqrt(var + self.eps)
        return self.gamma * x_hat + self.beta

In [5]:
d_model = 5
gamma1 = nn.Parameter(torch.zeros(d_model))
gamma2 = torch.zeros(d_model)
print(f"gamma1:\n{gamma1}")
print(f"gamma2:\n{gamma2}")
print(f"---------------------------")
matrix = torch.zeros(3,4)
print(matrix)

gamma1:
Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)
gamma2:
tensor([0., 0., 0., 0., 0.])
---------------------------
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


In [6]:
# Tạo input gồm 2 câu batch_size = 2
batch_size = 2
seq_len = 3
d_model = 5
x = torch.randn(batch_size,seq_len, d_model)
print(x)
mean = x.mean(dim=-1, keepdim=True)
print(f"Trung bình của từng hàng: \n{mean}")
var = x.var(dim = -1, keepdim=True, unbiased=False)
print(f"Phương sai của từng hàng:\n{var}")

tensor([[[-0.6834,  1.3955, -0.9725,  0.1299,  0.7456],
         [-1.0866,  1.3077,  0.6375,  0.0081,  0.5483],
         [-0.3463,  1.8357, -0.2784,  1.1115,  0.9505]],

        [[ 1.3233, -0.7076, -3.0274, -1.1843,  0.7891],
         [-1.2732,  0.2700, -0.0144, -0.6544,  1.3708],
         [-1.0561, -0.6736,  1.3640, -1.2083, -1.4339]]])
Trung bình của từng hàng: 
tensor([[[ 0.1230],
         [ 0.2830],
         [ 0.6546]],

        [[-0.5614],
         [-0.0602],
         [-0.6016]]])
Phương sai của từng hàng:
tensor([[[0.7714],
         [0.6395],
         [0.7127]],

        [[2.3733],
         [0.7966],
         [1.0272]]])


In [7]:
gamma = torch.randn(d_model)
print(f"gamma:\n{gamma}")
x_hat = (x - mean) / torch.sqrt(var + 1e-6)
print(f"Chuẩn hóa các giá trị của input:\n{x_hat}")
ans = gamma * x_hat + gamma
print(f"Đầu ra layernorm của input:\n{ans}")

gamma:
tensor([-0.3801, -1.2661,  0.1803, -0.0439, -0.2416])
Chuẩn hóa các giá trị của input:
tensor([[[-0.9181,  1.4488, -1.2473,  0.0079,  0.7088],
         [-1.7127,  1.2814,  0.4433, -0.3438,  0.3318],
         [-1.1856,  1.3990, -1.1051,  0.5411,  0.3505]],

        [[ 1.2234, -0.0949, -1.6007, -0.4043,  0.8767],
         [-1.3590,  0.3700,  0.0514, -0.6657,  1.6033],
         [-0.4485, -0.0710,  1.9393, -0.5986, -0.8212]]])
Đầu ra layernorm của input:
tensor([[[-0.0311, -3.1005, -0.0446, -0.0443, -0.4128],
         [ 0.2709, -2.8886,  0.2602, -0.0288, -0.3217],
         [ 0.0705, -3.0375, -0.0190, -0.0677, -0.3262]],

        [[-0.8451, -1.1459, -0.1083, -0.0262, -0.4533],
         [ 0.1364, -1.7346,  0.1896, -0.0147, -0.6288],
         [-0.2096, -1.1762,  0.5300, -0.0176, -0.0432]]])


In [ ]:
# Sublayer: Feed forward network: 2 hidden layer
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(d_ff, d_model)
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        output = self.linear2(x)
        return output

In [9]:
# Residual connection Layernorm
class SublayerConnection(nn.Module):
    # y = layernorm(x + sublayer(x))
    def __init__(self, d_model, dropout=0.1):
        super().__init__()
        self.norm = LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x, sublayer):
        output = self.norm(x + self.dropout(sublayer(x)))
        return output